<a href="https://colab.research.google.com/github/jhonmunevar/DesafioHV/blob/main/Desafio2hv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import re
import json
import spacy
from transformers import pipeline
import pdfplumber



In [5]:
# Cargar el modelo de spaCy (NPL) para procesamiento de texto en inglés con analisis gramatical
# en_core_web_sm Carga un modelo preentrenado de procesamiento de texto en ingles sm, modelo pequeño en tamaño y velocidad
# en_core_web_sm divide texto en palabras, reduce palabras a su forma base walk, Reconoce entindades nombradas (nombres, lugraes, etc)
nlp = spacy.load("en_core_web_sm")

# Inicializar el modelo para reconocimiento de entidades con Hugging Face Transformers
# Modelo LLM de Hugging Face Transformers) BERT. pipeline facilita el uso de modelos preentrenados de NLP,  ner es el tipo de tarea a realizar, en este caso reconocimiento de entidades nombradas NER
# Ambos modelos trabajan en conjunto para proporcionar un análisis más completo del texto en las hojas de vida.
nlp_ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
# Funciones para la extración de la información de las hojas de vida
def extraer_texto_pdf(ruta_pdf):
    """Extrae texto de un archivo PDF."""
    texto = ""
    with pdfplumber.open(ruta_pdf) as pdf:
        for pagina in pdf.pages:
            texto += pagina.extract_text()
    return texto

def extraer_datos_advanced(texto_cv):
    """Extrae nombre, email y teléfono usando NLP y LLMs."""
    doc = nlp(texto_cv)
    entidades = nlp_ner(texto_cv)

    # Extracción de nombre
    nombres = [ent.text for ent in doc.ents if ent.label_ == 'PERSON']
    nombre = nombres[0] if nombres else None

    # Extracción de email
    email = re.search(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', texto_cv)

    # Extracción de teléfono
    telefono = re.search(r'\+?\d[\d -]{8,15}\d', texto_cv)

    return nombre, email.group(0) if email else None, telefono.group(0) if telefono else None

def extraer_experiencia_advanced(texto_cv):
    """Extrae el número total de años de experiencia profesional."""
    experiencia = re.search(r'\b(\d+)\s+years?\b', texto_cv, re.IGNORECASE)
    return experiencia.group(1) if experiencia else None

def verificar_formacion_ia_advanced(texto_cv):
    """Verifica si el CV menciona formación en inteligencia artificial."""
    if re.search(r'\b(ai|artificial intelligence|machine learning)\b', texto_cv, re.IGNORECASE):
        return True
    return False

def generar_json(nombre, email, telefono, experiencia, formacion_ia):
    """Genera un JSON con los resultados de la extracción."""
    resultado = {
        "nombre": {"valor": nombre if nombre else None, "score": 1.0 if nombre else 0.0},
        "email": {"valor": email if email else None, "score": 1.0 if email else 0.0},
        "telefono": {"valor": telefono if telefono else None, "score": 1.0 if telefono else 0.0},
        "experiencia": {"valor": experiencia if experiencia else None, "score": 1.0 if experiencia else 0.0},
        "formacion_ia": {"valor": "Si" if formacion_ia else "No", "score": 1.0 if formacion_ia else 0.0}
    }
    return json.dumps(resultado, indent=4)


In [11]:
# Se Procesan las hojas de vida
# Directorio donde están las hojas de vida
directorio_cvs = '/content/sample_data/cvs'

# Lista para almacenar los resultados de todos los CVs
resultados_cvs = []

# Procesar cada archivo PDF en el directorio
for archivo_pdf in os.listdir(directorio_cvs):
    if archivo_pdf.endswith(".pdf"):
        ruta_pdf = os.path.join(directorio_cvs, archivo_pdf)

        # Extraer texto del CV
        texto_cv = extraer_texto_pdf(ruta_pdf)

        # Extraer la información del CV usando técnicas avanzadas
        nombre, email, telefono = extraer_datos_advanced(texto_cv)
        experiencia = extraer_experiencia_advanced(texto_cv)
        formacion_ia = verificar_formacion_ia_advanced(texto_cv)

        # Generar el resultado para este CV
        resultado = generar_json(nombre, email, telefono, experiencia, formacion_ia)
        resultados_cvs.append({
            "archivo": archivo_pdf,
            "resultado": json.loads(resultado)
        })

# Guardar todos los resultados en un archivo JSON consolidado
with open("/content/sample_data/resultados_cvs.json", "w") as json_file:
    json.dump(resultados_cvs, json_file, indent=4)

# Mostrar los resultados
print(json.dumps(resultados_cvs, indent=4))


[
    {
        "archivo": "1724342728_CV8.pdf",
        "resultado": {
            "nombre": {
                "valor": "Law",
                "score": 1.0
            },
            "email": {
                "valor": "drsantoshkakade@gmail.com",
                "score": 1.0
            },
            "telefono": {
                "valor": "09850671175 1495",
                "score": 1.0
            },
            "experiencia": {
                "valor": "20",
                "score": 1.0
            },
            "formacion_ia": {
                "valor": "No",
                "score": 0.0
            }
        }
    },
    {
        "archivo": "1724342738_CV9.pdf",
        "resultado": {
            "nombre": {
                "valor": "W. Fullerton Pkwy",
                "score": 1.0
            },
            "email": {
                "valor": "loren@shevitz.org",
                "score": 1.0
            },
            "telefono": {
                "valor": "773-665-1234",
   